In [ ]:
import torch as th 
from torch.utils.data import DataLoader
th.backends.cudnn.benchmark = True

In [ ]:
from networks import * 
from mainNet import *
from dataloader import CoCo
from lossFunction import Decoder_Train_Loss
import utils 

In [ ]:
from torch.utils.tensorboard import SummaryWriter

In [ ]:
import matplotlib.pyplot as plt
from PIL import Image
from torchvision import transforms

In [ ]:
train_net = Decoder_Train_Net(True).cuda().train()

In [ ]:
dataset = DataLoader(dataset= CoCo(r"./data\train2014", 1000), 
                     batch_size= 1, 
                     shuffle= True, 
                     num_workers= 4
)

In [ ]:
optimizer = th.optim.Adam(train_net.decoder.parameters(), lr= 0.001)

In [ ]:
scheduler = th.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode= 'min', 
                                                    factor= 0.9, patience= 10, 
                                                    verbose= True, min_lr= 1e-4)

In [ ]:
loss_fn = Decoder_Train_Loss().cuda()
# loss_fn = nn.L1Loss(reduction= 'sum').cuda()

In [ ]:
epochs = 3

In [ ]:
now_epoch: int = 1

In [ ]:
train_net.decoder.load_state_dict(th.load(r"./pretrained_models\decoder0.4.pth"))

In [ ]:
# writer = SummaryWriter(utils.get_logPath(r"./logs"))

In [ ]:
len_of_dataset = dataset.__len__()

In [ ]:
for _ in range(epochs): 
    loss_all = 0 
    print("epoch: {:d}".format(now_epoch))
    for img in dataset:
        img = img.cuda()
        train_net(img)
        
        loss = loss_fn(train_net.dlist ,train_net.elist)
        
        loss_all += loss.item()

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    # scheduler.step(loss_all)
    # writer.add_scalar("loss", loss_all, now_epoch)
    print("loss: ", loss_all / len_of_dataset)
    now_epoch += 1

In [ ]:
for name, parms in train_net.decoder.named_parameters():
	print('-->name:', name, '--weight', th.mean(parms.data), ' -->grad_value:', th.mean(parms.grad))

In [ ]:
for name, parms in train_net.decoder.named_parameters():
    print(parms.grad)

In [ ]:
for name, parms in train_net.named_parameters():
    print('-->grads', parms.requires_grad)

In [ ]:
tran = transforms.Compose([
    transforms.ToTensor(), 
    transforms.Normalize(mean= [0.4, 0.4, 0.4], std= [0.2, 0.2, 0.2])
])

In [ ]:
img = Image.open(r"data\test\Luana.png").convert('RGB')
img = tran(img)
img = img.unsqueeze_(0).cuda()
output = train_net(img)
re_img = utils.img_transfer(output, (0.2, 0.2, 0.2), (0.4, 0.4, 0.4))

In [ ]:
plt.figure('Image')
plt.imshow(re_img)
plt.axis('off')
plt.title('Luana')
plt.show()

In [ ]:
th.save(train_net.decoder.state_dict(), r"./pretrained_models/decoder0.4.pth")

In [ ]:
train_net.decoder.state_dict()